# 1. Thông tin chung
Thành viên:

|MSSV|Họ tên|
|----|-----|
|19120080|Lê Đức Huy|
|19120298|Mai Duy Nam|

# 2. Thông tin về dữ liệu

## 2.1 Thông tin chung
Dataset trong đồ án này được lấy từ Kaggle tại [đây](https://www.kaggle.com/halhuynh/it-jobs-dataset).

Dataset này chứa thông tin về các công ty IT ở Việt Nam và các công việc được đăng tải bởi các công ty này. Dữ liệu này được tác giả crawl từ trang web [ITviec](https://itviec.com/). Dataset này bao gồm 2 file:
* `companies.csv` chứa thông tin về các công ty (tên công ty, lượt rating, lượt review, địa chỉ, v.v.) có tăng tải việc làm trên ITviec
* `jobs.csv` chứa thông tin về việc làm được đăng tải bởi các công ty (tên công việc, mô tả, yêu cầu kỹ năng, v.v.)

Hai tập dữ liệu này có mối liên hệ với nhau thông qua cột `company_id`.

## 2.2 Khám phá dữ liệu

Ở các phần sau, ta lần lượt khám phá 2 bộ dữ liệu `companies` và `jobs` thu thập được

### 2.2.1 `companies`

#### Nạp dữ liệu

In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [55]:
companies_df = pd.read_csv('dataset/companies.csv')
companies_df.head()

,company_id,company_name,average_rating,num_review,city,type,num_employee,country,working_day,OT,overview,expertise,benifit,logo_link
0,kms-technology,KMS Technology,4.0,197 Reviews,"Ho Chi Minh, Da Nang, Ha Noi",Outsourcing,1000+,United States,Monday - Friday,No OT,"Established in 2009, KMS Technology is a U.S.-...",NaN,Want to be the Best? Join KMS!\n\n\nIt’s posit...,https://itviec.com/rails/active_storage/repres...
1,nab-in-collaboration-with-positive-thinking-co...,NAB in collaboration with Positive Thinking Co...,4.4,11 Reviews,Ho Chi Minh,Product,501-1000,Australia,Monday - Friday,No OT,NAB in collaboration with Positive Thinking Co...,Languages &amp; Frameworks\n\n\nJavaScript ES6...,THE BENEFITS AND PERKS\n\n\n\n\nVery competiti...,https://itviec.com/rails/active_storage/repres...
2,robert-bosch-engineering-and-business-solutions,Robert Bosch Engineering And Business Solutions,3.2,217 Reviews,"Ho Chi Minh, Ha Noi",Product,1-50,Germany,Monday - Friday,No OT,Robert BOSCH Engineering and Business Solution...,Embedded Software for Automotive Applications\...,Why \nBOSCH\n?\n\nBecause we don't just follow...,https://itviec.com/rails/active_storage/repres...
3,gft-technologies-vietnam,GFT Technologies Vietnam,NaN,Reviews,"Ho Chi Minh, Ha Noi",Outsourcing,51-150,Germany,Monday - Friday,No OT,Working at GFT Vietnam\n\n\nGFT is driving the...,"Technical frameworks\n\n\nJava/Kotlin, Spring ...",HR benefits\n\n\n\n\nCompetitive salary\n\n\nS...,https://itviec.com/rails/active_storage/repres...
4,tyme,Tyme,4.6,29 Reviews,"Ho Chi Minh, Ha Noi",Product,151-300,Vietnam,Monday - Friday,No OT,Tyme Vietnam\n is the international arm of \nT...,Mobile/ Tablet Development: \nNative Android/i...,Our goal is simple: to be the #1 digital techn...,https://itviec.com/rails/active_storage/repres...


#### Số lượng dòng và cột

In [56]:
print('Number of rows: {}\nNumber of columns: {}'.format(*companies_df.shape))

Number of rows: 2041
Number of columns: 14


#### Các dòng có ý nghĩa gì?
Mỗi dòng chứa thông tin của một công ty dựa theo trang ITviec. Mỗi công ty có một ID duy nhất.

Kiểm tra có dòng nào bị lặp hay không.

In [57]:
companies_df.duplicated().sum() != 0

False

Kiểm tra có ID công ty nào bị lặp hay không.

In [58]:
companies_df.duplicated('company_id').sum() != 0

False

#### Các cột có ý nghĩa gì?
Tác giả của bộ dữ liệu không mô tả chi tiết về ý nghĩa của các một mà chỉ nói "The column names are self-explained". Do đó ta phải tự suy luận về ý nghĩa của các cột:
* `company_id`: ID của công ty
* `company_name`: tên công ty
* `average_rating`: số điểm rating trung bình của công ty trên thang điểm 5
* `num_review`: số lượt review công ty
* `city`: thành phố
* `type`: loại hình công ty (outsourcing hay product)
* `num_employee`: số lượng nhân viên
* `country`: quốc gia
* `working_day`: ngày làm việc trong tuần
* `OT`: các chính sách về làm thêm giờ
* `overview`: tổng quan về công ty
* `expertise`: lĩnh vực chuyên ngành
* `benifit` (sai chính tả, phải là `benefit`): lợi ích khi tham gia công ty
* `logo_link`: đường dẫn đến logo công ty

Có một vấn đề về ý nghĩa các cột: cột `city` không thống nhất với cột `country` (có công ty có thành phố là `Ho Chi Minh` nhưng quốc gia là `Australia`). Do các công ty có thể có nhiều trụ sở đặt ở nhiều quốc gia, trong cùng quốc gia cũng được đặt ở nhiều thành phố khác nhau, cho nên trong các bước phân tích tiếp theo ta sẽ giả sử hai cột `city` và `country` có ý nghĩa như sau:
* Cột `city` chứa các thành phố ở Việt Nam nơi các văn phòng của công ty được đặt.
* Cột `country` chứa quốc gia nơi trụ sở chính của công ty được đặt.

Các cột còn lại đều có ý nghĩa rõ ràng, không có gì gây khó hiểu.

Trước khi đến bước phân tích tiếp theo, ta thực hiện một số thay đổi về các cột như sau:
* Đổi tên các cột cho hợp lý:
    * Đổi tên cột `benifit` bị sai chính tả
    * Đổi tên cột `city`, `num_review`, `num_employee` và `working_day` thành số nhiều
    * Đổi tên cột `OT` thành `overtime` để thống nhất với các tên cột còn lại
* Loại bỏ cột `logo_link` do không cần thiết

In [59]:
companies_df.rename(columns={'benifit': 'benefit',
                             'city': 'cities',
                             'num_review': 'num_reviews',
                             'num_employee': 'num_employees',
                             'working_day': 'working_days',
                             'OT': 'overtime'}, inplace=True)
companies_df.drop(columns='logo_link', inplace=True)

#### Mỗi cột có kiểu dữ liệu thế nào và có phù hợp để xử lý tiếp không?

In [60]:
companies_df.dtypes

company_id         object
company_name       object
average_rating    float64
num_reviews        object
cities             object
type               object
num_employees      object
country            object
working_days       object
overtime           object
overview           object
expertise          object
benefit            object
dtype: object

Có hai cột có thể phải cân nhắc chuyển về dạng số là `num_reviews` và `num_employees`. Các cột khác đều đã có kiểu dữ liệu phù hợp.

Đối với cột `num_employees`, dữ liệu của ta được thu thập theo dạng một khoảng giá trị. Việc chuyển đổi cột này về dạng số hay không phụ thuộc vào phân tích nào mà ta đang thực hiện. Do đó tạm thời ở bước này ta không thay đổi cột `num_employees`.

In [61]:
companies_df.num_employees.unique()

array(['1000+', '501-1000', '1-50', '51-150', '151-300', '301-500'],
      dtype=object)

Đối với `num_reviews`, kiểm tra các giá trị unique của nó, ta thấy các giá trị đều có dạng: `"<một số> Review[s]"`, ngoại trừ giá trị duy nhất không tuân theo pattern này là `Reviews`. Vì vậy, ta có thể lấy ra giá trị số, lọc bỏ chuỗi ` Reviews` dư thừa và chuyển kiểu dữ liệu cột thành dạng float. Đối với giá trị `Reviews`, ta gán bằng NaN.

In [62]:
companies_df.num_reviews.unique()

array(['197 Reviews', '11 Reviews', '217 Reviews', 'Reviews',
       '29 Reviews', '16 Reviews', '8 Reviews', '60 Reviews', '9 Reviews',
       '127 Reviews', '6 Reviews', '34 Reviews', '3 Reviews',
       '20 Reviews', '185 Reviews', '1486 Reviews', '57 Reviews',
       '22 Reviews', '68 Reviews', '1 Review', '21 Reviews', '12 Reviews',
       '23 Reviews', '7 Reviews', '17 Reviews', '5 Reviews', '96 Reviews',
       '37 Reviews', '207 Reviews', '109 Reviews', '4 Reviews',
       '33 Reviews', '76 Reviews', '18 Reviews', '10 Reviews',
       '2 Reviews', '65 Reviews', '15 Reviews', '41 Reviews',
       '19 Reviews', '27 Reviews', '13 Reviews', '26 Reviews',
       '42 Reviews', '183 Reviews', '24 Reviews', '54 Reviews',
       '14 Reviews', '38 Reviews', '451 Reviews', '28 Reviews',
       '36 Reviews', '52 Reviews', '31 Reviews', '92 Reviews',
       '30 Reviews', '86 Reviews', '32 Reviews', '25 Reviews',
       '43 Reviews', '46 Reviews', '97 Reviews', '44 Reviews',
       '226 Revi

In [63]:
import re
def get_num_reviews(x):
    result = re.match('^(\d+) Reviews?$', x)
    return float(result.group(1)) if result != None else np.nan
companies_df.num_reviews = companies_df.num_reviews.map(get_num_reviews, na_action='ignore')
companies_df.num_reviews.unique()

array([1.970e+02, 1.100e+01, 2.170e+02,       nan, 2.900e+01, 1.600e+01,
       8.000e+00, 6.000e+01, 9.000e+00, 1.270e+02, 6.000e+00, 3.400e+01,
       3.000e+00, 2.000e+01, 1.850e+02, 1.486e+03, 5.700e+01, 2.200e+01,
       6.800e+01, 1.000e+00, 2.100e+01, 1.200e+01, 2.300e+01, 7.000e+00,
       1.700e+01, 5.000e+00, 9.600e+01, 3.700e+01, 2.070e+02, 1.090e+02,
       4.000e+00, 3.300e+01, 7.600e+01, 1.800e+01, 1.000e+01, 2.000e+00,
       6.500e+01, 1.500e+01, 4.100e+01, 1.900e+01, 2.700e+01, 1.300e+01,
       2.600e+01, 4.200e+01, 1.830e+02, 2.400e+01, 5.400e+01, 1.400e+01,
       3.800e+01, 4.510e+02, 2.800e+01, 3.600e+01, 5.200e+01, 3.100e+01,
       9.200e+01, 3.000e+01, 8.600e+01, 3.200e+01, 2.500e+01, 4.300e+01,
       4.600e+01, 9.700e+01, 4.400e+01, 2.260e+02, 1.790e+02, 6.600e+01,
       4.500e+01, 4.900e+01, 5.800e+01, 2.320e+02, 8.700e+01])

#### Với mỗi cột kiểu dữ liệu dạng số, các giá trị được phân bố như thế nào?

In [64]:
numerical_columns = ['average_rating', 'num_reviews']
categorical_columns = companies_df.columns.to_list()
categorical_columns.remove('average_rating')
categorical_columns.remove('num_reviews')

In [65]:
companies_df[numerical_columns].describe()

,average_rating,num_reviews
count,563.000000,869.000000
mean,3.987034,12.989643
std,0.597378,57.202636
min,2.000000,1.000000
25%,3.600000,2.000000
50%,4.000000,4.000000
75%,4.400000,10.000000
max,5.000000,1486.000000


In [66]:
companies_df[numerical_columns].isna().sum() / companies_df[numerical_columns].shape[0] * 100

average_rating    72.415483
num_reviews       57.422832
dtype: float64

Có thể thấy, số lượng giá trị thiếu của các cột số là khá lớn. Còn lại, các số liệu thống kê khác đều ổn, không có điều gì bất thường.

#### Với mỗi cột dạng phân loại, các giá trị được phân bố thế nào

In [67]:
desc = companies_df[categorical_columns].describe()
desc

,company_id,company_name,cities,type,num_employees,country,working_days,overtime,overview,expertise,benefit
count,2041,2041,2041,2041,2041,2041,2026,1865,2040,477,1576
unique,2041,2041,71,2,6,39,2,4,2022,474,1573
top,kms-technology,KMS Technology,"District 1, Ho Chi Minh",Product,1-50,Vietnam,Monday - Friday,No OT,.,"C++, VC++, C# .NET Framework Web Framework Jav...",Salary: Competitive (based on skills and exper...
freq,1,1,190,1534,1054,1202,1855,1589,12,2,2


Tỷ lệ giá trị thiếu

In [68]:
(1 - desc.loc['count'] / companies_df.shape[0]) * 100

company_id             0.0
company_name           0.0
cities                 0.0
type                   0.0
num_employees          0.0
country                0.0
working_days      0.734934
overtime          8.623224
overview          0.048996
expertise        76.629103
benefit           22.78295
Name: count, dtype: object

Một số giá trị ngẫu nhiên

In [69]:
companies_df[categorical_columns].sample(8)

,company_id,company_name,cities,type,num_employees,country,working_days,overtime,overview,expertise,benefit
1389,skylink-group,Skylink Group,"District 2, Ho Chi Minh",Product,301-500,Vietnam,Monday - Friday,No OT,Skylink Group có mặt trên thị trường vào năm 2...,NaN,"Môi trường làm việc mở, thoải mái, khuyến khíc..."
1611,ezm,EZM,"Thanh Xuan, Ha Noi",Product,51-150,Vietnam,Monday - Friday,No OT,"EZM chuyên thiết kế đồ họa và hoạt động, phát ...",NaN,Chế độ lương thưởng:\n\n\n- Mức lương: 18.000...
1458,edso-labs,Edso Labs,"Cau Giay, Ha Noi",Product,51-150,Vietnam,Monday - Friday,No OT,We create an internal culture of innovation an...,"Agile, scrum\n\n\nTechstack: .NET, Vue js, Jav...",Attractive salary and package\n\n\nWorking tim...
1175,colibri-growth,Colibri Growth,"Ho Chi Minh, Ha Noi",Product,1-50,Singapore,Monday - Friday,No OT,Colibri is all about growth. Tech startups in ...,Each of our teams focuses on different product...,We are an organisation of talented (product) d...
1674,trung-tam-kinh-doanh-vnpt-ha-noi,Trung tâm kinh doanh VNPT - Hà Nội,"Hoan Kiem, Ha Noi",Product,1000+,Vietnam,Monday - Saturday,No OT,Trung tâm kinh doanh VNPT – Hà Nội là chi nhán...,NaN,Ứng viên trúng tuyển được ký hợp đồng lao động...
1711,vina-chg,Vina CHG,"District 12, Ho Chi Minh",Product,1-50,Vietnam,Monday - Saturday,No OT,"Được thành lập từ 2008, với sứ mệnh đặt ra từ ...",NaN,"Thưởng lương tháng 13, thưởng các ngày lễ, tết..."
1699,admon-co-ltd,ADMON Co.ltd,"Binh Thanh, Ho Chi Minh",Product,1-50,Vietnam,Monday - Friday,No OT,.,NaN,NaN
25,contemi-vietnam,Contemi Vietnam,"District 3, Ho Chi Minh",Product,51-150,Norway,Monday - Friday,No OT,"You probably know many IT companies from US, J...",NaN,Our benefits package includes:\n\n\n\n\nAttrac...


Có thể thấy dữ liệu dạng phân loại khá đầy đủ (ngoại trừ cột `expertise` số lượng giá trị thiếu lên đến 70%).

### 2.2.2 `jobs`

#### Nạp dữ liệu

In [70]:
jobs_df = pd.read_csv('dataset/jobs.csv')
jobs_df.head()

,job_id,company_id,job_name,taglist,location,three_reasons,description
0,kms-technology:jrsr_qa_engineer_kms_labs_bonus,kms-technology,"(Jr/Sr) QA Engineer, KMS Labs - BONUS",QA QC English Tester,"290/26 Nam Ky Khoi Nghia, ward 8, District 3, ...",Inspired Leadership Team and Talented Colleagu...,KMS Labs is the startup incubation arm of KMS ...
1,kms-technology:engineering_manager_bonus,kms-technology,Engineering Manager - BONUS,Project Manager Agile English,"02 Tan Vien, ward 2, Tan Binh, Ho Chi Minh",Inspired Leadership Team and Talented Colleagu...,Full 13th Month Salary. ***Apply and Join in S...
2,kms-technology:fullstack_mobile_mobilenodejs_k...,kms-technology,"Fullstack Mobile (Mobile,NodeJs) Kobiton",NodeJS React Native Fresher Accepted,"02 Tan Vien, ward 2, Tan Binh, Ho Chi Minh",Inspired Leadership Team and Talented Colleagu...,is the mobile experience platform trusted by l...
3,kms-technology:jrsrprincipal_java_developer_bonus,kms-technology,(Jr/Sr/Principal) Java Developer- BONUS,Java JavaScript Spring,"174 Thai Ha, Dong Da, Ha Noi",Inspired Leadership Team and Talented Colleagu...,Full 13th Month Salary ***Apply and Join in Se...
4,kms-technology:product_manager_kms_labs_bonus,kms-technology,"Product Manager, KMS Labs - BONUS",Product Manager English Manager,"290/26 Nam Ky Khoi Nghia, ward 8, District 3, ...",Inspired Leadership Team and Talented Colleagu...,KMS Labs\n is the startup incubation arm of KM...


#### Số lượng dòng và cột

In [71]:
print('Number of rows: ', jobs_df.shape[0])
print('Number of columns: ', jobs_df.shape[1])

Number of rows:  1412
Number of columns:  7


#### Mỗi dòng có ý nghĩa gì ?

Mỗi dòng là thông tin của một công việc đang được đăng tin tuyển dụng trên ITViec.

Mỗi dòng được phân biệt bởi `job_id`

#### Dữ liệu có bị lặp không?

In [72]:
jobs_df.duplicated().sum() != 0

False

Như vậy dữ liệu không bị trùng lặp giữa các dòng.

Kiểm tra cột `job_id` có bị lặp không?

In [73]:
jobs_df.duplicated('job_id').sum() != 0

True

Như vậy ở cột `job_id` có dữ liệu trùng lặp. Ta sẽ kiểm tra tiếp ở dưới.

Trích xuất các dòng có dữ liệu lặp:

In [74]:
jobs_df[jobs_df.duplicated('job_id', keep=False)]

,job_id,company_id,job_name,taglist,location,three_reasons,description
301,houze-group:backend_lead_pythongolangnodejs,houze-group,Backend Lead ( Python/Golang/NodeJS),NodeJS Python Golang,"114-116 Ngô Quyền, Phường 8, District 5, Ho Ch...",Attractive salary and benefits package\nCareer...,"Define, code, test, and deploy new features to..."
303,houze-group:backend_lead_pythongolangnodejs,houze-group,Backend Lead ( Python/Golang/NodeJS/...),Team Leader Python Golang,"114-116 Ngô Quyền, Phường 8, District 5, Ho Ch...",Attractive salary and benefits package\nCareer...,"Define, code, test, and deploy new features to..."


Như vậy theo quan sát, ta thấy giữa 2 dòng chỉ khác biệt ở cột `taglist`, tất cả các cột còn lại đều giống nhau.

Ở đây cả bộ dữ liệu chỉ duy nhất 1 `job_id` bị trùng lặp. Để tiện xử lý ta sẽ loại bỏ 1 hàng (cụ thể hàng cuối).

In [75]:
jobs_df = jobs_df.drop_duplicates('job_id', keep = 'first')

Số lượng dòng sau khi xử lý:

In [53]:
print('Number of rows: ', len(jobs_df))

Number of rows:  1411


#### Mỗi cột có ý nghĩa gì?

Bởi dữ liệu được thu thập trên Kaggle, tuy nhiên tác giả không cung cấp thông tin về ý nghĩa của các cột, chính vì thế, dựa vào ý nghĩa của dữ liệu ta sẽ định nghĩa các cột như sau:
   * `job_id`: ID của công việc
   * `company_id`: ID của công ty đăng tuyển công việc
   * `job_name`: tên công việc
   * `taglist`: danh sách tag liên quan đến công việc
   * `location`: địa điểm làm việc
   * `three_reasons`: phúc lợi/lý do nên làm việc ở đây
   * `description`: mô tả công việc

#### Mỗi cột có kiểu dữ liệu gì? Có phù hợp để xử lý tiếp không?

In [79]:
jobs_df.dtypes

job_id           object
company_id       object
job_name         object
taglist          object
location         object
three_reasons    object
description      object
dtype: object

Như đã thấy tất cả thông tin về công việc không có cột nào nên đưa về dạng số nên ta không cần xử lý gì thêm ở bước này, tất cả dữ liệu đã có kiểu dữ liệu phù hợp để xử lý.

#### Phân tích các cột dữ liệu dạng số

Trong các cột dữ liệu trên, không có cột dữ liệu nào dạng số nên ta không thể thống kê các giá trị được.

#### Với mỗi cột dang phân loại, các giá trị được phân bố như thế nào?

In [81]:
statistic = jobs_df.describe()
statistic

,job_id,company_id,job_name,taglist,location,three_reasons,description
count,1411,1411,1411,1411,1411,1097,1411
unique,1411,682,1358,923,696,492,1408
top,kms-technology:jrsr_qa_engineer_kms_labs_bonus,vietnam-international-bank-vib,"Frontend Dev (JavaScript, HTML5, CSS)",JavaScript HTML5 CSS,"Sailing Tower Building, District 1, Ho Chi Minh",Global Exposure\nFast Track Career\nDiversifie...,Asilla là một startup công nghệ tiên phong phá...
freq,1,27,8,20,19,17,2


Số lượng giá trị thiếu ở mỗi cột:

In [85]:
miss_info = len(jobs_df) - statistic.loc['count']
miss_info

job_id             0
company_id         0
job_name           0
taglist            0
location           0
three_reasons    314
description        0
Name: count, dtype: object

Tỉ lệ giá trị thiếu:

In [87]:
miss_info/len(jobs_df) * 100

job_id                 0.0
company_id             0.0
job_name               0.0
taglist                0.0
location               0.0
three_reasons    22.253721
description            0.0
Name: count, dtype: object

Như đã thống kê, đa phần các cột có đầy đủ các dữ liệu trừ cột `three_reason` tỉ lệ thiếu lên tới 22,25\%.
Nhìn vào hàng `unique`, đa phần có giá trị khá lớn, cho thấy được sự đa dạng của việc tuyển dụng ở lĩnh vực IT.

Một số giá trị ngẫu nhiên:

In [100]:
jobs_df.sample(10)

,job_id,company_id,job_name,taglist,location,three_reasons,description
1279,finviet:nodejs_backend_dev_javascript_1800,finviet,NodeJS Backend Dev (JavaScript) ~ 1800$,NodeJS HTML5 JavaScript,"34 Hoang Viet, Tan Binh, Ho Chi Minh",A fast growing start-up company in FIN TECH\nC...,"Design, develop, improve, and maintain core an..."
104,vietnam-international-bank-vib:digital_data_sc...,vietnam-international-bank-vib,Digital Data Scientist,MySQL Python,"Sailing Tower Building, District 1, Ho Chi Minh",NaN,* Mục đích công việc:\n\n\nChịu trách nhiệm th...
789,flynk:midsr_backend_developer_net_c,flynk,"Mid/Sr. Backend Developer (.NET, C#)",C# English .NET,"Bcons Tower, Nguyen Van Thuong, Binh Thanh, Ho...",We believe you are our most valuable asset\nEx...,We are recruiting Backend Developer. Details a...
153,vng-corporation:leadsenior_qa_engineer_auto_za...,vng-corporation,Lead/Senior QA Engineer (Auto) - ZaloPay,Tester Agile Java,"VNG Campus, Tân Thuận Đông, District 7, Ho Chi...",Attractive salary & benefits you'll love\nBuil...,Working closely with the manual members to get...
751,port-cities-vietnam:project_manager_vietnam,port-cities-vietnam,Project Manager Vietnam,Project Manager ERP Business Analyst,"H3 building, District 4, Ho Chi Minh",Working on big B2B project for interesting cli...,We are hiring a Project Manager/ ERP Consultan...
168,fpt-software:sap_functional_consultant_no_prob...,fpt-software,SAP Functional Consultant - NO PROBATION,SAP System Engineer English,"FPT Software Đường D1 Phường Tân Phú , Distric...",Global Exposure\nFast Track Career\nDiversifie...,"Special offers: No Probation, get sign on bonu..."
141,skedulo-vietnam:partner_enablement_program_man...,skedulo-vietnam,Partner Enablement Program Manager,Manager English Project Manager,"An Phú Plaza, 117-119 Ly Chinh Thang street, w...",Sign on bonus to all level (*)\nGreat healthca...,The Global Partner Delivery team is looking fo...
159,niteco-vietnam-co-ltd:senior_project_managers_x6,niteco-vietnam-co-ltd,Senior Project Managers (x6),Project Manager Agile English,"14th Floor, CLand Tower, 156 Xa Dan II, Dong D...",Attractive salary package\nPremium healthcare ...,We are looking for experienced Project Manager...
1310,cong-ty-co-phan-cong-nghe-dng:php_developer_ja...,cong-ty-co-phan-cong-nghe-dng,"PHP Developer (JavaScript, HTML5)",PHP HTML5 JavaScript,"84 Nguyen Hao Vinh, Tan Quy Ward, Tan Phu, Ho ...",NaN,"Tham gia phát triển CRM bằng PHP, mysql. \n\n\..."
53,tyme:business_analyst_manager_agile,tyme,Business Analyst Manager (Agile),Business Analyst Agile Manager,"HIU Tower, 215 Điện Biên Phủ, Phường 15, Binh ...",Excellent environment and team to help you gro...,Business Analyst Manager is responsible for le...


Dữ liệu cho thấy được không có sự bất thường ở bộ dữ liệu này.
Tuy nhiên dữ liệu ở 2 cột `three_reasons` và `description` có các dòng dữ liệu thuộc 2 ngôn ngữ khác nhau (Việt và Anh) có thể gây khó khăn trong việc phân tích.

# 3. Câu hỏi cần trả lời

## 3.1 Câu hỏi 1

### 3.1.1 Nêu vấn đề

### 3.1.2 Data Preprocessing

### 3.1.3 Analysis, Visualization

### 3.1.4 Kết luận

## 3.2 Câu hỏi 2

### 3.2.1 Nêu vấn đề

### 3.2.2 Data Preprocessing

### 3.2.3 Analysis, Visualization

### 3.2.4 Kết luận

## 3.3 Phương hướng phát triển nếu còn thời gian

# 4. Cảm nhận sau khi kết thúc môn học

## 4.1 Lê Đức Huy - 19120080

## 4.2 Mai Duy Nam - 19120298

# 5. Tài liệu tham khảo